# Holder Wallet Conformance Flows (v3) - CTWalletCrossDeferred

# 0.0 Initial setup

## 0.1 Setup conformance

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [2]:
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK

key_did = KeyDid()

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbqWZDHTMcqPRofEnj7mZksq5gnFqR3QCDYY6EBdV9bwY8DwXXaGHh86Bse8WiUiGUt13yyt5d9c5qQMzFy8MgufY1oqSvwDve43jaPMCRkqnzWtyRSCbcrBA1c3DcART22p


## 1.1 Initiate Credential Issuance

In [3]:
from ebsi_wallet.util import parse_query_string_parameters_from_url
from ebsi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type =  CredentialTypes.CTWalletCrossDeferred
qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)


[21:12:05] Credential offer URI:                                                                   ]8;id=722404;file:///tmp/ipykernel_54302/3425914532.py\3425914532.py]8;;\:]8;id=879158;file:///tmp/ipykernel_54302/3425914532.py#15\15]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/0ddd6dcd-f47d-4ac2-a5                 
           59-1ab25eda04c2                                                                                         

[21:12:06] Credential offer:                                                                       ]8;id=535512;file:///tmp/ipykernel_54302/3425914532.py\3425914532.py]8;;\:]8;id=447733;file:///tmp/ipykernel_54302/3425914532.py#18\18]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossDeferred'                                                                 
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'authorization_code': {                                                                         
                       'issuer_state':                                                                             
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYnFXWkRIVE1jcVBSb2ZFbmo3bVprc3E1Z25GcVIzUUNEWVk2RUJkVjlid1k4R                 
           HdYWGFHSGg4NkJzZThXaVVpR1V0MTN5eXQ1ZDljNXFRTXpGeThNZ3VmWTFvcVN2d0R2ZTQzamFQTUNSa3Fueld0                 
           eVJTQ2JjckJBMWMzRGNBUlQyMnAiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                 
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl0sImlhdCI6MTY4NzE4OTMyNS                 
           wiZXhwIjoxNjg3MTg5NjI1LCJpc3MiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hb                 
           mNlL3YzL2lzc3Vlci1tb2NrIiwiYXVkIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3Jt                 
           YW5jZS92My9hdXRoLW1vY2siLCJzdWIiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW9                 
           5dHlrVVozZXlxaHQxajlLYnFXWkRIVE1jcVBSb2ZFbmo3bVprc3E1Z25GcVIzUUNEWVk2RUJkVjlid1k4RHdYWG                 
           FHSGg4NkJzZThXaVVpR1V0MTN5eXQ1ZDljNXFRTXpGeThNZ3VmWTFvcVN2d0R2ZTQzamFQTUNSa3Fueld0eVJTQ                 
           2JjckJBMWMzRGNBUlQyMnAifQ.yhHSNZqb8gcuOIOcjlujyVHFNzNyzdulKPzC8FXddQW90ir1MCZizA4YltUtr                 
           MY2XKE-K7t4MY1sIylswQpVLg'    

           Credential issuer configuration:                                                        ]8;id=586826;file:///tmp/ipykernel_54302/3425914532.py\3425914532.py]8;;\:]8;id=59423;file:///tmp/ipykernel_54302/3425914532.py#21\21]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework': 

[21:12:07] Authorization server configuration:                                                     ]8;id=37200;file:///tmp/ipykernel_54302/3425914532.py\3425914532.py]8;;\:]8;id=239958;file:///tmp/ipykernel_54302/3425914532.py#24\24]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Perform authorization request and obtain ID token request

In [4]:
from ebsi_wallet.siop_auth.util import (
    perform_authorization, 
    AuthorizationRequestQueryParams,
    get_authorization_response_query_params,
    generate_code_challenge,
    generate_code_verifier
)
import uuid
import json

state = str(uuid.uuid4())
nonce = str(uuid.uuid4())
code_verifier = generate_code_verifier()

authorization_details = [{"type":"openid_credential","format":"jwt_vc","types":credential_offer.credentials[0].get("types"),"locations": ["https://api-conformance.ebsi.eu/conformance/v3/issuer-mock"]}]
redirect_uri = "http://localhost:8080"
client_metadata = {"vp_formats_supported":{"jwt_vp":{"alg":["ES256"]},"jwt_vc":{"alg":["ES256"]}},"response_types_supported":["vp_token","id_token"],"authorization_endpoint":redirect_uri}
authorization_request_query_params = AuthorizationRequestQueryParams(
    response_type="code",
    scope="openid",
    state=state,
    client_id=key_did._did,
    authorization_details=json.dumps(authorization_details, separators=(',', ':')),
    redirect_uri=redirect_uri,
    nonce=nonce,
    code_challenge=generate_code_challenge(code_verifier),
    code_challenge_method="S256",
    client_metadata=json.dumps(client_metadata, separators=(',', ':')),
    issuer_state=credential_offer.grants.get("authorization_code").get("issuer_state")
)

auth_resp = await perform_authorization(auth_server_configuration.authorization_endpoint, authorization_request_query_params)
auth_resp_uri = str(auth_resp).split("Location': '")[1].split("'")[0]
console.log("Authorization response URI: ", auth_resp_uri)
auth_resp_query_params = get_authorization_response_query_params(auth_resp_uri)
console.log("Authorization response query params: ", auth_resp_query_params)

[21:12:08] Authorization response URI:                                                             ]8;id=892013;file:///tmp/ipykernel_54302/1836098038.py\1836098038.py]8;;\:]8;id=863076;file:///tmp/ipykernel_54302/1836098038.py#34\34]8;;\
           http://localhost:8080?state=0f771943-0fe0-46ff-a93c-fbe562753dfa&client_id=https%3A%2F%                 
           2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock&redirect_uri=https%3A%2F%2Fapi                 
           -conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Fdirect_post&response_type=id_toke                 
           n&response_mode=direct_post&scope=openid&nonce=9d263cdd-392a-40d9-9f12-5e6d501c65f2&req                 
           uest_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Frequest                 
           _uri%2Fc3e4d89c-baa7-4d72-aede-4ed80d653681                                                             

           Authorization response query params:                                                    ]8;id=707474;file:///tmp/ipykernel_54302/1836098038.py\1836098038.py]8;;\:]8;id=514515;file:///tmp/ipykernel_54302/1836098038.py#36\36]8;;\
           AuthorizationResponseQueryParams(                                                                       
               state='0f771943-0fe0-46ff-a93c-fbe562753dfa',                                                       
               client_id='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                               
               redirect_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                 
           ,                                                                                                       
               response_type='id_token',                                                                           
               response_mode='direct_post',                                                                        
               scope='openid',                                                                                     
               nonce='9d263cdd-392a-40d9-9f12-5e6d501c65f2',                                                       
               request_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/request_uri/c                 
           3e4d89c-baa7-4d72-aede-4ed80d653681'                                                                    
           )                                                                                                       

## 1.3 Send ID token

In [5]:
from ebsi_wallet.siop_auth.util import send_id_token_response

id_token = key_did.generate_id_token(auth_server_uri=auth_resp_query_params.client_id, nonce=auth_resp_query_params.nonce)
auth_code_response = await send_id_token_response(auth_resp_query_params.redirect_uri, id_token, auth_resp_query_params.state)
auth_code_response = str(auth_code_response).split("Location': '")[1].split("'")[0]
state = parse_query_string_parameters_from_url(auth_code_response).get("state")[0]
auth_code = parse_query_string_parameters_from_url(auth_code_response).get("code")[0]
console.log("Authorization code: ", auth_code)

[21:12:09] Authorization code:  e1bf6958-6d8f-4f0a-8809-0d024ef2e28c                                ]8;id=685567;file:///tmp/ipykernel_54302/1546471797.py\1546471797.py]8;;\:]8;id=397484;file:///tmp/ipykernel_54302/1546471797.py#8\8]8;;\

## 1.4 Exchange code for access token

In [6]:
from ebsi_wallet.siop_auth.util import exchange_auth_code_for_access_token

token_uri = auth_resp_query_params.client_id + "/token"
access_token_response = await exchange_auth_code_for_access_token(token_uri, key_did._did, auth_code, code_verifier)
console.log("Access token response: ", access_token_response)

           Access token response:                                                                    ]8;id=243659;file:///tmp/ipykernel_54302/474231906.py\474231906.py]8;;\:]8;id=968232;file:///tmp/ipykernel_54302/474231906.py#5\5]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWV               
           ZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImExYmExOWM3LWZiODItNDAzMy1hMTdhLTJhNzIwN               
           WNhZTA3MCIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudG               
           lhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV               
           1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVy               
           aWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl19XSwiY19ub25jZSI6ImUxMWRmMGM2L               
           TZhYzAtNDkzZS1hMTFkLTI5ODRhMGY3NDJkZCIsImNfbm9uY2VfZXhwaXJlc19pbiI6ODY0MDAsImNsaWVudF9pZC               
           I6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticVdaREhUTWN               
           xUFJvZkVuajdtWmtzcTVnbkZxUjNRQ0RZWTZFQmRWOWJ3WThEd1hYYUdIaDg2QnNlOFdpVWlHVXQxM3l5dDVkOWM1               
           cVFNekZ5OE1ndWZZMW9xU3Z3RHZlNDNqYVBNQ1JrcW56V3R5UlNDYmNyQkExYzNEY0FSVDIycCJ9LCJpc3MiOiJod               
           HRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2F1dGgtbW9jayIsImF1ZCI6WyJodH               
           RwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInN1YiI6ImR               
           pZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticVdaREhUTWNxUFJv               
           ZkVuajdtWmtzcTVnbkZxUjNRQ0RZWTZFQmRWOWJ3WThEd1hYYUdIaDg2QnNlOFdpVWlHVXQxM3l5dDVkOWM1cVFNe               
           kZ5OE1ndWZZMW9xU3Z3RHZlNDNqYVBNQ1JrcW56V3R5UlNDYmNyQkExYzNEY0FSVDIycCIsImlhdCI6MTY4NzE4OT               
           MyOSwiZXhwIjoxNjg3Mjc1NzI5fQ.ylvuOv4-Qm8ewmpzinXGgh1BX1YKAc63RxLNK_2c7bXSZqRg9r7-Sxqen01H               
           l3prF_UZLl0Ws8pSMRincWa6TA',                                                                            
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWl               
           GbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjJlYWQzYmFiLTZlMTItNDRiMC04MDQ4LTE2N2FkZmRkN               
           zViNCIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2               
           NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J               
           xV1pESFRNY3FQUm9mRW5qN21aa3NxNWduRnFSM1FDRFlZNkVCZFY5YndZOER3WFhhR0hoODZCc2U4V2lVaUdVdDEz               
           eXl0NWQ5YzVxUU16Rnk4TWd1Zlkxb3FTdndEdmU0M2phUE1DUmtxbnpXdHlSU0NiY3JCQTFjM0RjQVJUMjJwIiwiY               
           XVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JxV1pESF               
           RNY3FQUm9mRW5qN21aa3NxNWduRnFSM1FDRFlZNkVCZFY5YndZOER3WFhhR0hoODZCc2U4V2lVaUdVdDEzeXl0NWQ               
           5YzVxUU16Rnk4TWd1Zlkxb3FTdndEdmU0M2phUE1DUmtxbnpXdHlSU0NiY3JCQTFjM0RjQVJUMjJwIiwiaWF0Ijox               
           Njg3MTg5MzI5LCJleHAiOjE2ODcxODk0NDl9.C0Mctf_5YF9AcKeOmmWu1fgjYHJcJd4YravR3jfNkECODY-F3ZAn               
           WCMqlVzEjrCWNn89xgSHCBYeJNqIOeiX8Q',                                                                    
               c_nonce='e11df0c6-6ac0-493e-a11d-2984a0f742dd',                                                     
               c_nonce_expires_in=86400     

## 1.5 Request credential to get acceptance token to initiate deferred flow 

In [7]:
from ebsi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
acceptance_token_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Acceptance token response: ", acceptance_token_response)## 1.6 Request credential using deferred endpoint

           Credential request JWT:                                                                   ]8;id=338418;file:///tmp/ipykernel_54302/380321948.py\380321948.py]8;;\:]8;id=441834;file:///tmp/ipykernel_54302/380321948.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVW               
           jNleXFodDFqOUticVdaREhUTWNxUFJvZkVuajdtWmtzcTVnbkZxUjNRQ0RZWTZFQmRWOWJ3WThEd1hYYUdIaDg2Qn               
           NlOFdpVWlHVXQxM3l5dDVkOWM1cVFNekZ5OE1ndWZZMW9xU3Z3RHZlNDNqYVBNQ1JrcW56V3R5UlNDYmNyQkExYzN               
           EY0FSVDIycCN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JxV1pESFRNY3FQ               
           Um9mRW5qN21aa3NxNWduRnFSM1FDRFlZNkVCZFY5YndZOER3WFhhR0hoODZCc2U4V2lVaUdVdDEzeXl0NWQ5YzVxU               
           U16Rnk4TWd1Zlkxb3FTdndEdmU0M2phUE1DUmtxbnpXdHlSU0NiY3JCQTFjM0RjQVJUMjJwIiwidHlwIjoib3Blbm               
           lkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h               
           bmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg3Mjc1NzI5LCJpYXQiOjE2ODcxODkzMjksImlzcyI6ImRpZDprZ               
           Xk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticVdaREhUTWNxUFJvZkVuaj               
           dtWmtzcTVnbkZxUjNRQ0RZWTZFQmRWOWJ3WThEd1hYYUdIaDg2QnNlOFdpVWlHVXQxM3l5dDVkOWM1cVFNekZ5OE1               
           ndWZZMW9xU3Z3RHZlNDNqYVBNQ1JrcW56V3R5UlNDYmNyQkExYzNEY0FSVDIycCIsIm5vbmNlIjoiZTExZGYwYzYt               
           NmFjMC00OTNlLWExMWQtMjk4NGEwZjc0MmRkIn0.nlBZI_pVvPv1R--wwLqA5LpiuX1J5JTLdzHXhuV4XHJsZGaqW               
           Nhdn-LR1IOF2JUFbkdyDg3wDnZ0zbC_tql2aw                                                                   

[21:12:10] Acceptance token response:                                                                ]8;id=338766;file:///tmp/ipykernel_54302/380321948.py\380321948.py]8;;\:]8;id=435814;file:///tmp/ipykernel_54302/380321948.py#6\6]8;;\
           {'acceptance_token': 'OIFRg7ByryL1UiFbt-LbIKd6izujyurEfMqIGF9Vk9U'}                                     

## 1.6 Request credential using deferred endpoint

In [8]:
from ebsi_wallet.siop_auth.util import send_deferred_credential_request
from time import sleep

console.log("Waiting for 5 seconds...")
sleep(5)
console.log("Sending deferred credential request...")
credential_response = await send_deferred_credential_request(credential_issuer_configuration.deferred_credential_endpoint, acceptance_token_response.get("acceptance_token"))
console.log("Credential response: ", credential_response)

           Waiting for 5 seconds...                                                                 ]8;id=127173;file:///tmp/ipykernel_54302/3408621820.py\3408621820.py]8;;\:]8;id=141461;file:///tmp/ipykernel_54302/3408621820.py#4\4]8;;\

[21:12:15] Sending deferred credential request...                                                   ]8;id=734145;file:///tmp/ipykernel_54302/3408621820.py\3408621820.py]8;;\:]8;id=109547;file:///tmp/ipykernel_54302/3408621820.py#6\6]8;;\

[21:12:16] Credential response:                                                                     ]8;id=285066;file:///tmp/ipykernel_54302/3408621820.py\3408621820.py]8;;\:]8;id=729683;file:///tmp/ipykernel_54302/3408621820.py#8\8]8;;\
           {                                                                                                       
               'format': 'jwt_vc',                                                                                 
               'credential':                                                                                       
           'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWl                
           kI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNv                
           bmZvcm1hbmNlI2YzZjEwNGRkLTFjN2YtNDMwNC04YzBiLWU0MmRlMTExY2YyOCIsInN1YiI6ImRpZDprZXk6ejJk                
           bXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticVdaREhUTWNxUFJvZkVuajdtWmtz                
           cTVnbkZxUjNRQ0RZWTZFQmRWOWJ3WThEd1hYYUdIaDg2QnNlOFdpVWlHVXQxM3l5dDVkOWM1cVFNekZ5OE1ndWZZ                
           MW9xU3Z3RHZlNDNqYVBNQ1JrcW56V3R5UlNDYmNyQkExYzNEY0FSVDIycCIsImlzcyI6ImRpZDplYnNpOnpoSkFS                
           alBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg3MTg5MzMwLCJleHAiOjE2ODcyNzU3MzAsImlhdCI6MTY4NzE4                
           OTMzMCwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwi                
           aWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2YzZjEwNGRkLTFjN2YtNDMwNC04YzBiLWU0MmRlMTExY2YyOCIsInR5                
           cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldENyb3Nz                
           RGVmZXJyZWQiXSwiaXNzdWVyIjoiZGlkOmVic2k6emhKQVJqUE42OWNFdGdQeEhlbjFNaWQiLCJpc3N1YW5jZURh                
           dGUiOiIyMDIzLTA2LTE5VDE1OjQyOjEwWiIsImlzc3VlZCI6IjIwMjMtMDYtMTlUMTU6NDI6MTBaIiwidmFsaWRG                
           cm9tIjoiMjAyMy0wNi0xOVQxNTo0MjoxMFoiLCJleHBpcmF0aW9uRGF0ZSI6IjIwMjMtMDYtMjBUMTU6NDI6MTBa                
           IiwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1Bn                
           WW95dHlrVVozZXlxaHQxajlLYnFXWkRIVE1jcVBSb2ZFbmo3bVprc3E1Z25GcVIzUUNEWVk2RUJkVjlid1k4RHdY                
           WGFHSGg4NkJzZThXaVVpR1V0MTN5eXQ1ZDljNXFRTXpGeThNZ3VmWTFvcVN2d0R2ZTQzamFQTUNSa3Fueld0eVJT                
           Q2JjckJBMWMzRGNBUlQyMnAifSwiY3JlZGVudGlhbFNjaGVtYSI6eyJpZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1h                
           bmNlLmVic2kuZXUvdHJ1c3RlZC1zY2hlbWFzLXJlZ2lzdHJ5L3YyL3NjaGVtYXMvejNNZ1VGVWtiNzIydXE0eDNk                
           djV5QUptbk5tekRGZUs1VUM4eDgzUW9lTEpNIiwidHlwZSI6IkZ1bGxKc29uU2NoZW1hVmFsaWRhdG9yMjAyMSJ9                
           LCJ0ZXJtc09mVXNlIjp7ImlkIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS90cnVzdGVkLWlzc3Vl                
           cnMtcmVnaXN0cnkvdjQvaXNzdWVycy9kaWQ6ZWJzaTp6aEpBUmpQTjY5Y0V0Z1B4SGVuMU1pZC9hdHRyaWJ1dGVz                
           L2YxYWNiMzc1ZmNmYmUxMWE2MDAzZDViNzEwYmYzN2VjYjI3ODJjMTFhNGU0Mjg1NzAxMzM3NzMyYzI2NjRkMDAi                
           LCJ0eXBlIjoiSXNzdWFuY2VDZXJ0aWZpY2F0ZSJ9fX0.Q4aj5my0Y838hR4qycX5qu9BIyoWweRR4xIiWiOWFaLD                
           uQnWzIOQPfLM-pHLrrOLcFE_EYfFaA908szrXNItaQ'                                                             
           }                                                                                                       